# 第五章 开始学习pandas

`Numpy`主要处理同质数字数组类型，而pandas处理表格或者异质数据。

In [1]:
import numpy as np
import pandas as pd

In [2]:
from pandas import Series, DataFrame

## 5.1 pandas数据结构简介
两个主要干活的数据结构`Series`和`DataFrame`。

### Series
序列是一维数组，包括同样类型的值序列，还有相关的数据标签数组，被称作`index`。
最简单的序列仅由一个数据数组组成

In [3]:
import numpy as np
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
pd.options.display.max_columns = 20
pd.options.display.max_colwidth = 80
np.set_printoptions(precision=4, suppress=True)

In [4]:
obj = pd.Series([4, 7, -5, 3])
obj

0    4
1    7
2   -5
3    3
dtype: int64

In [5]:
obj.array

<PandasArray>
[4, 7, -5, 3]
Length: 4, dtype: int64

In [6]:
obj.index

`.array`属性的结果是`PandasArray`，通常是个NumPy数组的封装，但也可能包括特殊扩展的数据类型。参见7.23，page 224.

创建序列的时候，你经常向在索引中用标记区分各个数据点：

In [8]:
obj2 = pd.Series([4, 7, -5, 3], index=["d", "b", "a", "c"])
obj2

d    4
b    7
a   -5
c    3
dtype: int64

In [9]:
obj2.index

Index(['d', 'b', 'a', 'c'], dtype='object')

In [10]:
obj2["a"]

-5

In [11]:
obj2["d"] = 6

In [12]:
obj2[["c", "a", "d"]]

c    3
a   -5
d    6
dtype: int64

用NumPy函数或者类似NumPy的操作，比如用布尔数组过滤、标量乘法或者应用数学函数，会保留索引到值的联系。

In [13]:
obj2[obj2 > 0]

d    6
b    7
c    3
dtype: int64

In [14]:
obj2 * 2

d    12
b    14
a   -10
c     6
dtype: int64

In [15]:
import numpy as np
np.exp(obj2)

d     403.428793
b    1096.633158
a       0.006738
c      20.085537
dtype: float64

序列也可以看作固定长度、有序的字典。

In [16]:
"b" in obj2

True

In [17]:
"e" in obj2

False

可以从字典生成序列

In [18]:
sdata = {"Ohio": 35000, "Texas": 71000, "Oregon": 16000, "Utah": 5000}
obj3 = pd.Series(sdata)
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

可以用`to_dic`方法转换成字典。

In [19]:
obj3.to_dict()

{'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}

传字典的时候，序列会保持键的顺序，这和字典插入的顺序有关，可以传入想要的顺序改变缺省顺序。

In [20]:
states = ["California", "Ohio", "Oregon", "Texas"]
obj4 = pd.Series(sdata, index=states)
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

In [13]:
这里加州没有数，而Utah不在states中，因此被扔掉了。

In [21]:
pd.isna(obj4)

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [22]:
pd.notna(obj4)

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

In [23]:
obj4.isna()

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

我会在第七章讨论丢失的数据。

序列的一个有用特色是它在算数操作的时候会自动按标签对齐。

In [29]:
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

In [30]:
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

In [27]:
obj3 + obj4

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

数据对齐在后面会详细讨论。
如果你对数据库由经验，可以将此看作类似并操作。

系列对象自身及其索引都有一个`name`属性，该属性和pandas功能的其他区域集成：

In [31]:
obj4.name = "population"
obj4.index.name = "state"
obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

In [17]:
系列的索引可以通过赋值在位改变“

In [32]:
obj

0    4
1    7
2   -5
3    3
dtype: int64

In [33]:
obj.index = ["Bob", "Steve", "Jeff", "Ryan"]
obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

### DataFrame
DataFrame表示一个矩形数据表，包括列的有序命名集合，每一列值的类型可以不同。
DataFrame既有行索引，也有列索引，可以被看作共享同一索引的序列字典。

有多种方法可以构建DataFrame，最常见的是从等长度的列表或者Numpy数组的字典构建。

In [34]:
data = {"state": ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada", "Nevada"],
        "year": [2000, 2001, 2002, 2001, 2002, 2003],
        "pop": [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
frame = pd.DataFrame(data)

生成的Dataframe会自动指定索引，和Series一样。列的顺序按照`data`里键的顺序排列（取决于在字典里插入的顺序）

In [35]:
frame

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


对大的DataFrame，`head`只选择前五行，`tail`返回后五行

In [36]:
frame.head()

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9


In [37]:
frame.tail()

,state,year,pop
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


如果指定了列的顺序，DataFrame的列会按照该顺序排列。

In [38]:
pd.DataFrame(data, columns=["year", "state", "pop"])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9
5,2003,Nevada,3.2


如果传递了地点里没有的列，结果是没有的值

In [40]:
frame2 = pd.DataFrame(data, columns=["year", "state", "pop", "debt"])
frame2

,year,state,pop,debt
0,2000,Ohio,1.5,NaN
1,2001,Ohio,1.7,NaN
2,2002,Ohio,3.6,NaN
3,2001,Nevada,2.4,NaN
4,2002,Nevada,2.9,NaN
5,2003,Nevada,3.2,NaN


In [41]:
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

DataFrame中的列可以用类似字典的方法或者点属性的方法作为Series取出。

In [42]:
frame2["state"]

0      Ohio
1      Ohio
2      Ohio
3    Nevada
4    Nevada
5    Nevada
Name: state, dtype: object

In [43]:
frame2.year

0    2000
1    2001
2    2002
3    2001
4    2002
5    2003
Name: year, dtype: int64

`frame[column]`可以用于任何列名，但`fram2.column`只能用于列名是合法变量名的列。

注意返回的序列其索引和DataFrame一样，他们的`name`属性也被合理设置。

行可以用特别的`iloc`和`loc`属性按照位置和名称取出。

In [44]:
frame2.loc[1]

year     2001
state    Ohio
pop       1.7
debt      NaN
Name: 1, dtype: object

In [45]:
frame2.iloc[2]

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: 2, dtype: object

可以通过赋值修改列。下面给空白的`debt`列夫之

In [46]:
frame2["debt"] = 16.5
frame2

,year,state,pop,debt
0,2000,Ohio,1.5,16.5
1,2001,Ohio,1.7,16.5
2,2002,Ohio,3.6,16.5
3,2001,Nevada,2.4,16.5
4,2002,Nevada,2.9,16.5
5,2003,Nevada,3.2,16.5


In [47]:
frame2["debt"] = np.arange(6.)
frame2

,year,state,pop,debt
0,2000,Ohio,1.5,0.0
1,2001,Ohio,1.7,1.0
2,2002,Ohio,3.6,2.0
3,2001,Nevada,2.4,3.0
4,2002,Nevada,2.9,4.0
5,2003,Nevada,3.2,5.0


用列表或者数组给列表赋值的时候，值的长度必须和Data Frame匹配。
如果指定了序列，其标签会按照DataFrame索引重新排列，在没有值的任何索引的值上出插入没有的值。

In [48]:
val = pd.Series([-1.2, -1.5, -1.7], index=["two", "four", "five"])
frame2["debt"] = val
frame2

,year,state,pop,debt
0,2000,Ohio,1.5,NaN
1,2001,Ohio,1.7,NaN
2,2002,Ohio,3.6,NaN
3,2001,Nevada,2.4,NaN
4,2002,Nevada,2.9,NaN
5,2003,Nevada,3.2,NaN


给不存在的列赋值会创建新列，`del`关键字会删除列

In [52]:
frame2["eastern"] = frame2["state"] == "Ohio"
frame2

,year,state,pop,debt,eastern
0,2000,Ohio,1.5,NaN,True
1,2001,Ohio,1.7,NaN,True
2,2002,Ohio,3.6,NaN,True
3,2001,Nevada,2.4,NaN,False
4,2002,Nevada,2.9,NaN,False
5,2003,Nevada,3.2,NaN,False


用`frame2.eastern`点属性方法不能创建新列。

In [53]:
del frame2["eastern"]
frame2.columns

索引DataFrame返回的列是底层数据的视图而不是拷贝。
这样对序列的改动会反应在DataFrame上。

嵌套的字典的字典是另外一种方式：

In [54]:
populations = {"Ohio": {2000: 1.5, 2001: 1.7, 2002: 3.6},
               "Nevada": {2001: 2.4, 2002: 2.9}}

In [55]:
frame3 = pd.DataFrame(populations)
frame3

,Ohio,Nevada
2000,1.5,NaN
2001,1.7,2.4
2002,3.6,2.9


转置

In [57]:
frame3.T

,2000,2001,2002
Ohio,1.5,1.7,3.6
Nevada,NaN,2.4,2.9


In [58]:
pd.DataFrame(populations, index=[2001, 2002, 2003])

,Ohio,Nevada
2001,1.7,2.4
2002,3.6,2.9
2003,NaN,NaN


In [59]:
pdata = {"Ohio": frame3["Ohio"][:-1],
         "Nevada": frame3["Nevada"][:2]}
pd.DataFrame(pdata)

,Ohio,Nevada
2000,1.5,NaN
2001,1.7,2.4


In [60]:
frame3.index.name = "year"
frame3.columns.name = "state"
frame3

state,Ohio,Nevada
year,,
2000,1.5,NaN
2001,1.7,2.4
2002,3.6,2.9


In [61]:
frame3.to_numpy()

array([[1.5, nan],
       [1.7, 2.4],
       [3.6, 2.9]])

In [62]:
frame2.to_numpy()

array([[2000, 'Ohio', 1.5, nan],
       [2001, 'Ohio', 1.7, nan],
       [2002, 'Ohio', 3.6, nan],
       [2001, 'Nevada', 2.4, nan],
       [2002, 'Nevada', 2.9, nan],
       [2003, 'Nevada', 3.2, nan]], dtype=object)

### Index对象
pandas的索引对象负责保留轴标签（包括DataFrame的列名称）和其他宏数据（包括轴的名称）。任何你构建系列或者DataFrame时使用的标签数组或者其他序列都在内部被转化为索引。

In [63]:
obj = pd.Series(np.arange(3), index=["a", "b", "c"])
index = obj.index
index

Index(['a', 'b', 'c'], dtype='object')

In [64]:
index[1:]

Index(['b', 'c'], dtype='object')

Index对象是不可变的，因此用户不能修改

In [65]:
labels = pd.Index(np.arange(3))
labels

Int64Index([0, 1, 2], dtype='int64')

In [66]:
obj2 = pd.Series([1.5, -2.5, 0], index=labels)
obj2

0    1.5
1   -2.5
2    0.0
dtype: float64

In [67]:
obj2.index is labels

True

除了和数组相似，index表现得也象一个固定储存的集合。

In [68]:
frame3

state,Ohio,Nevada
year,,
2000,1.5,NaN
2001,1.7,2.4
2002,3.6,2.9


In [69]:
frame3.columns

Index(['Ohio', 'Nevada'], dtype='object', name='state')

In [70]:
"Ohio" in frame3.columns

True

In [71]:
2003 in frame3.index

False

和集合不一样，pandas的index可以包含重复标签

In [72]:
pd.Index(["foo", "foo", "bar", "bar"])

Index(['foo', 'foo', 'bar', 'bar'], dtype='object')

选择重复的标签会选择所有出现的标签。

表5-2 一些索引方法和性质

|方法/性质|说明|
|---|---|
|append()|和另外的索引对象连接，生成新的索引|
|difference()|作为索引计算集合差|
|intersecction()|计算交集|
|union|计算并|
|isin()|计算表示传入的集合中是否包括各个值的布尔数组|
|delete()|删除index i计算新索引|
|drop()|删除传入的值计算新索引|
|insert()|在index i插入元素计算新索引|
|is_monotonic|如果每个元素都大于或者等于前一个元素返回`True`|
|is_unique|如果索引没有重复值返回`True`|
|unique()|计算索引中没有重复值的数组|

## 5.2 基本功能
#### Reindexing
创建新对象，重新排列值使其和新索引匹配。

In [73]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=["d", "b", "a", "c"])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

在本序列上调用`reindex`会根据新索引重新排列数据，如果有之前没有的索引，会引入没有值。

In [74]:
obj2 = obj.reindex(["a", "b", "c", "d", "e"])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

对时间序列这样的有序数据，你可能想在重新索引的时候进行插值或者填入住。`method`可选参数让你可以这么做。`ffill`向前填充值

In [75]:
obj3 = pd.Series(["blue", "purple", "yellow"], index=[0, 2, 4])
obj3

0      blue
2    purple
4    yellow
dtype: object

In [76]:
obj3.reindex(np.arange(6), method="ffill")

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

在DataFrame上，`reindex`可以改变（行）索引、列，还可以同时改变二者。
如果值传入一个序列，结果会重新排序行

In [77]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)),
                     index=["a", "c", "d"],
                     columns=["Ohio", "Texas", "California"])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [78]:
frame2 = frame.reindex(index=["a", "b", "c", "d"])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


可以用`columns`关键字重新索引列

In [79]:
states = ["Texas", "Utah", "California"]
frame.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


在特定轴上mreindex的另一个方法是传入新的axis标签作为位置参数，然后用axis参数指定要reindex的轴

In [80]:
frame.reindex(states, axis="columns")

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


表 5-3 `reindex`函数的参数


你还可以loc操作符reindex。这个只在所有新标签都存在的时候才可以

In [81]:
frame.loc[["a", "d", "c"], ["California", "Texas"]]

,California,Texas
a,2,1
d,8,7
c,5,4


### 从坐标轴去除项

In [82]:
obj = pd.Series(np.arange(5.), index=["a", "b", "c", "d", "e"])
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [83]:
new_obj = obj.drop("c")
new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [84]:
obj.drop(["d", "c"])

a    0.0
b    1.0
e    4.0
dtype: float64

在DataFrame中，可以从任何轴上删除索引值。

In [85]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=["Ohio", "Colorado", "Utah", "New York"],
                    columns=["one", "two", "three", "four"])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [86]:
data.drop(index=["Colorado", "Ohio"])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [87]:
data.drop(columns=["two"])

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [88]:
data.drop("two", axis=1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [89]:
data.drop(["two", "four"], axis="columns")

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


### 索引、选择和过滤

系列索引（`obj[...]`）和NumPy数组的索引类似，但除了用整数外，你还可以用许略的索引值。

In [90]:
obj = pd.Series(np.arange(4.), index=["a", "b", "c", "d"])
obj
obj["b"]
obj[1]
obj[2:4]
obj[["b", "a", "d"]]
obj[[1, 3]]
obj[obj < 2]

a    0.0
b    1.0
dtype: float64

虽然可以用这种方法通过标记选组数据，最好还是用特殊的`loc`运算符选择索引值

In [91]:
obj.loc[["b", "a", "d"]]

b    1.0
a    0.0
d    3.0
dtype: float64

优先采用`loc`的原因是用[]索引的时候对整数的处理不同。

In [56]:
obj1 = pd.Series([1, 2, 3], index=[2, 0, 1])
obj2 = pd.Series([1, 2, 3], index=["a", "b", "c"])
obj1
obj2
obj1[[0, 1, 2]]
obj2[[0, 1, 2]]

In [57]:
obj1.iloc[[0, 1, 2]]
obj2.iloc[[0, 1, 2]]

In [58]:
obj2.loc["b":"c"]

In [59]:
obj2.loc["b":"c"] = 5
obj2

In [60]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=["Ohio", "Colorado", "Utah", "New York"],
                    columns=["one", "two", "three", "four"])
data
data["two"]
data[["three", "one"]]

In [61]:
data[:2]
data[data["three"] > 5]

In [62]:
data < 5

In [63]:
data[data < 5] = 0
data

In [64]:
data
data.loc["Colorado"]

In [65]:
data.loc[["Colorado", "New York"]]

In [66]:
data.loc["Colorado", ["two", "three"]]

In [67]:
data.iloc[2]
data.iloc[[2, 1]]
data.iloc[2, [3, 0, 1]]
data.iloc[[1, 2], [3, 0, 1]]

In [68]:
data.loc[:"Utah", "two"]
data.iloc[:, :3][data.three > 5]

In [69]:
data.loc[data.three >= 2]

In [70]:
ser = pd.Series(np.arange(3.))
ser
ser[-1]

In [71]:
ser

In [72]:
ser2 = pd.Series(np.arange(3.), index=["a", "b", "c"])
ser2[-1]

In [73]:
ser.iloc[-1]

In [74]:
ser[:2]

In [75]:
data.loc[:, "one"] = 1
data
data.iloc[2] = 5
data
data.loc[data["four"] > 5] = 3
data

In [76]:
data.loc[data.three == 5]["three"] = 6

In [77]:
data

In [78]:
data.loc[data.three == 5, "three"] = 6
data

In [79]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=["a", "c", "d", "e"])
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1],
               index=["a", "c", "e", "f", "g"])
s1
s2

In [80]:
s1 + s2

In [81]:
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), columns=list("bcd"),
                   index=["Ohio", "Texas", "Colorado"])
df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)), columns=list("bde"),
                   index=["Utah", "Ohio", "Texas", "Oregon"])
df1
df2

In [82]:
df1 + df2

In [83]:
df1 = pd.DataFrame({"A": [1, 2]})
df2 = pd.DataFrame({"B": [3, 4]})
df1
df2
df1 + df2

In [84]:
df1 = pd.DataFrame(np.arange(12.).reshape((3, 4)),
                   columns=list("abcd"))
df2 = pd.DataFrame(np.arange(20.).reshape((4, 5)),
                   columns=list("abcde"))
df2.loc[1, "b"] = np.nan
df1
df2

In [85]:
df1 + df2

In [86]:
df1.add(df2, fill_value=0)

In [87]:
1 / df1
df1.rdiv(1)

In [88]:
df1.reindex(columns=df2.columns, fill_value=0)

In [89]:
arr = np.arange(12.).reshape((3, 4))
arr
arr[0]
arr - arr[0]

In [90]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),
                     columns=list("bde"),
                     index=["Utah", "Ohio", "Texas", "Oregon"])
series = frame.iloc[0]
frame
series

In [91]:
frame - series

In [92]:
series2 = pd.Series(np.arange(3), index=["b", "e", "f"])
series2
frame + series2

In [93]:
series3 = frame["d"]
frame
series3
frame.sub(series3, axis="index")

In [94]:
frame = pd.DataFrame(np.random.standard_normal((4, 3)),
                     columns=list("bde"),
                     index=["Utah", "Ohio", "Texas", "Oregon"])
frame
np.abs(frame)

In [95]:
def f1(x):
    return x.max() - x.min()

frame.apply(f1)

In [96]:
frame.apply(f1, axis="columns")

In [97]:
def f2(x):
    return pd.Series([x.min(), x.max()], index=["min", "max"])
frame.apply(f2)

In [98]:
def my_format(x):
    return f"{x:.2f}"

frame.applymap(my_format)

In [99]:
frame["e"].map(my_format)

In [100]:
obj = pd.Series(np.arange(4), index=["d", "a", "b", "c"])
obj
obj.sort_index()

In [101]:
frame = pd.DataFrame(np.arange(8).reshape((2, 4)),
                     index=["three", "one"],
                     columns=["d", "a", "b", "c"])
frame
frame.sort_index()
frame.sort_index(axis="columns")

In [102]:
frame.sort_index(axis="columns", ascending=False)

In [103]:
obj = pd.Series([4, 7, -3, 2])
obj.sort_values()

In [104]:
obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])
obj.sort_values()

In [105]:
obj.sort_values(na_position="first")

In [106]:
frame = pd.DataFrame({"b": [4, 7, -3, 2], "a": [0, 1, 0, 1]})
frame
frame.sort_values("b")

In [107]:
frame.sort_values(["a", "b"])

In [108]:
obj = pd.Series([7, -5, 7, 4, 2, 0, 4])
obj.rank()

In [109]:
obj.rank(method="first")

In [110]:
obj.rank(ascending=False)

In [111]:
frame = pd.DataFrame({"b": [4.3, 7, -3, 2], "a": [0, 1, 0, 1],
                      "c": [-2, 5, 8, -2.5]})
frame
frame.rank(axis="columns")

In [112]:
obj = pd.Series(np.arange(5), index=["a", "a", "b", "b", "c"])
obj

In [113]:
obj.index.is_unique

In [114]:
obj["a"]
obj["c"]

In [115]:
df = pd.DataFrame(np.random.standard_normal((5, 3)),
                  index=["a", "a", "b", "b", "c"])
df
df.loc["b"]
df.loc["c"]

In [116]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5],
                   [np.nan, np.nan], [0.75, -1.3]],
                  index=["a", "b", "c", "d"],
                  columns=["one", "two"])
df

In [117]:
df.sum()

In [118]:
df.sum(axis="columns")

In [119]:
df.sum(axis="index", skipna=False)
df.sum(axis="columns", skipna=False)

In [120]:
df.mean(axis="columns")

In [121]:
df.idxmax()

In [122]:
df.cumsum()

In [123]:
df.describe()

In [124]:
obj = pd.Series(["a", "a", "b", "c"] * 4)
obj.describe()

In [125]:
price = pd.read_pickle("examples/yahoo_price.pkl")
volume = pd.read_pickle("examples/yahoo_volume.pkl")

In [126]:
returns = price.pct_change()
returns.tail()

In [127]:
returns["MSFT"].corr(returns["IBM"])
returns["MSFT"].cov(returns["IBM"])

In [128]:
returns.corr()
returns.cov()

In [129]:
returns.corrwith(returns["IBM"])

In [130]:
returns.corrwith(volume)

In [131]:
obj = pd.Series(["c", "a", "d", "a", "a", "b", "b", "c", "c"])

In [132]:
uniques = obj.unique()
uniques

In [133]:
obj.value_counts()

In [134]:
pd.value_counts(obj.to_numpy(), sort=False)

In [135]:
obj
mask = obj.isin(["b", "c"])
mask
obj[mask]

In [136]:
to_match = pd.Series(["c", "a", "b", "b", "c", "a"])
unique_vals = pd.Series(["c", "b", "a"])
indices = pd.Index(unique_vals).get_indexer(to_match)
indices

In [137]:
data = pd.DataFrame({"Qu1": [1, 3, 4, 3, 4],
                     "Qu2": [2, 3, 1, 2, 3],
                     "Qu3": [1, 5, 2, 4, 4]})
data

In [138]:
data["Qu1"].value_counts().sort_index()

In [139]:
result = data.apply(pd.value_counts).fillna(0)
result

In [140]:
data = pd.DataFrame({"a": [1, 1, 1, 2, 2], "b": [0, 0, 1, 0, 0]})
data
data.value_counts()

In [142]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS